In [ ]:
# import all the needed libraries
import pandas as pd
from depiction.models.paccmann import PaccMannSmiles, PaccMannCellLine
from depiction.core import Task, DataType
from depiction.interpreters.uw_model import UWModel

## Data

In [ ]:
# Parse data from GDSC
# drugs
drugs = pd.read_csv(
    '../data/paccmann/gdsc.smi', sep='\t',
    index_col=1, header=None,
    names=['smiles']
)
# cell lines
cell_lines = pd.read_csv('../data/paccmann/gdsc.csv.gz', index_col=1)
# sensitivity data
drug_sensitivity = pd.read_csv('../data/paccmann/gdsc_sensitivity.csv.gz', index_col=0)

## Interpretability on the drug level for a cell line of interest

In [ ]:
task = Task.CLASSIFICATION
data_type = DataType.TEXT
class_names = ['Not Effective', 'Effective']

In [ ]:
interpreter = 'lime'
explanation_configs = {
    'labels': (1,),
}
interpreter_params = {
    'class_names': class_names,
    'split_expression': list,
    'bow': False
}
explainer = UWModel(interpreter, task, data_type, explanation_configs, **interpreter_params)

In [ ]:
selected_cell_line = 'NCI-H1648'
selected_drug_sensitivity = drug_sensitivity[drug_sensitivity['cell_line'] == selected_cell_line]

In [ ]:
classifier = PaccMannSmiles(cell_lines.loc[selected_cell_line][3:].values)

In [ ]:
explainer.interpret(classifier.predict, drugs.loc['Erlotinib'].item())